In [18]:
%autosave 30

Autosaving every 30 seconds


In [36]:
%load_ext autoreload
%autoreload 2

In [19]:
cd /home/huawei123/kwx1991442/tests-ui-classification

/home/huawei123/kwx1991442/tests-ui-classification


## Imports

In [20]:
import json
import os
import glob
from collections import namedtuple
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from typing import *

In [21]:
with open('config.json') as file:
    cfg = json.load(file)

# DATA_FOLDER = "data/rust/src/test/ui/"
DATA_FOLDER = cfg['DATA_FOLDER']
EXCLUDED_SUBDIRS = cfg['EXCLUDED_SUBDIRS']
CODEPARROT_FOLDER = cfg['CODEPARROT_FOLDER']

In [22]:
DATA_FOLDER, CODEPARROT_FOLDER

('data/rust/tests/ui/', 'data/codeparrot-small')

## Dataloader

In [23]:
Item = namedtuple("Item", "fname relpath")
"""Structure to store location of each file

fname (str) : filename without extension (filename.*)
relpath (str) : relative path to the file (tests/ui/{relpath}/filename.*)
"""

'Structure to store location of each file\n\nfname (str) : filename without extension (filename.*)\nrelpath (str) : relative path to the file (tests/ui/{relpath}/filename.*)\n'

SyntaxError: invalid syntax (4139311659.py, line 115)

In [25]:
train = UITestsDataset(mode='train')
infer = UITestsDataset(mode='infer')

In [195]:
len(train), len(infer)

(11327, 391)

In [196]:
len(train.classes), len(infer.classes)

(307, 1)

In [197]:
t = train
code = train[1][1]

In [26]:
train.tokenizer.vocab_size

32768

In [198]:
output[100:200] == train[1][1][100:200]

True

In [199]:
# train.visualize(output[149:160])
output[149:160]

':...) -> Va'

In [200]:
# train.visualize(train[1][1][149:161])
train[1][1][149:159]

':...) -> V'

In [201]:
train.encode(train[1][1][149:161])

tensor([[   26, 11965,  1035,   812,    65,    44]])

In [202]:
train.encode(output[149:160])

tensor([[   26, 11965,  1035,   812,    65]])

In [203]:
train.decode([19172]), train.decode([11965])

('...)', '...)')

In [204]:
# train.encode(' -> ')
# train.decode([1035]), train.decode([221])
# rawtext.split()

In [205]:
t = train

In [206]:
A = ': ...) -> '
B = ':...) -> '

In [207]:
t.encode(A), t.encode(B)

(tensor([[   26, 19172,  1035,   221]]),
 tensor([[   26, 11965,  1035,   221]]))

In [208]:
t.decode(t.encode(A)[0]), t.decode(t.encode(B)[0])

(':...) -> ', ':...) -> ')

In [209]:
t.decode(t.encode(A)[0]) == t.decode(t.encode(B)[0])

True

In [217]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(CODEPARROT_FOLDER)
model = AutoModelForCausalLM.from_pretrained(CODEPARROT_FOLDER)
model.train()
# model.resize_token_embeddings(len(tokenizer))

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32768, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=32768, bias=False)
)

In [218]:
input = t[0][0]
model.resize_token_embeddings(len(t.tokenizer))

Embedding(32867, 768)

In [219]:
outputs = model(input)

In [220]:
outputs.__dict__.keys()

dict_keys(['loss', 'logits', 'past_key_values', 'hidden_states', 'attentions', 'cross_attentions'])

In [221]:
print(t.decode(torch.argmax(outputs.logits, dim=2)[0]))


0]]dot]text"]

[cr_c)]intous=
#!#__ intC"
__c_(type(void
 use, int"_
#x: u,

 void#size

 u
:


} C_x(

pub unsafe extern "C" fn use_var_arg_lifetime(
:intusize,
: &
 &usize { //~
}y:}

#:0_]]
 argument specifier

x
x_${adic.integer.type.0
0:$:
//

(

 &usize { //}&

2_ size

}|
LL|@(
 is is return value
 the stringorrowed function
 and
 is no

 the
 be usedorrowed. the*[
 using the function--if' function parameter
=
( | ) -> &ustatic'size {
}=

+++
#+[
ing
 to a error

error example information, the function, see toerror--help`0106`




In [223]:
outputs

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[ 1.4998,  0.3302,  2.9010,  ..., -0.4366, -0.5687,  0.8230],
         [-1.0267,  6.2797,  3.4461,  ..., -1.8261,  0.1386, -0.0206],
         [-0.7753,  4.0988,  3.7241,  ..., -1.3609, -1.5311,  1.0269],
         ...,
         [ 0.8419,  3.8446,  3.3226,  ...,  0.1626, -1.2107,  0.8576],
         [ 4.1289,  4.2072,  3.6644,  ...,  0.5152,  0.2223,  0.8054],
         [ 7.2140,  4.6161,  6.0339,  ...,  0.9913, -1.0268,  0.2820]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[-1.9715e-01, -6.6485e-01,  8.1289e-01,  ...,  2.0410e-01,
            1.2667e+00, -2.4230e-01],
          [ 1.1905e-01,  3.5118e-01, -1.9434e+00,  ...,  4.9683e-01,
           -1.1296e+00, -2.3965e-01],
          [-6.6278e-01,  5.8562e-01, -6.7710e-01,  ..., -6.4066e-01,
           -1.3822e-01,  7.6050e-01],
          ...,
          [ 2.7407e-01,  4.3489e-01, -1.3391e+00,  ..., -2.6701e-01,
           -3.4000e-01,  7.0858e-01],
      

In [27]:
s = train[1][1]

In [30]:
tok = t.tokenizer

In [31]:
tok(s)

{'input_ids': [3168, 59, 1556, 323, 63, 466, 275, 298, 773, 937, 199, 3168, 59, 889, 63, 1516, 61, 199, 3168, 59, 4445, 8, 67, 63, 3808, 31764, 1874, 199, 199, 1180, 6170, 447, 11286, 447, 91, 29933, 1296, 12, 812, 65, 1296, 4748, 12084, 199, 199, 5645, 19293, 21815, 298, 35, 2, 4325, 949, 63, 3765, 16, 28, 7, 70, 30, 1547, 26, 399, 890, 12, 2771, 26, 11965, 1035, 812, 65, 1296, 4748, 28, 7, 70, 30, 469, 272, 2771, 272, 5001, 94, 62, 12539, 26, 30097, 1443, 440, 12166, 1846, 8321, 272, 5001, 94, 92, 12539, 26, 30097, 1443, 440, 12166, 1846, 8321, 199, 93, 199, 199, 5645, 19293, 21815, 298, 35, 2, 4325, 949, 63, 3765, 17, 1547, 26, 399, 890, 12, 2771, 26, 11965, 1035, 812, 65, 1296, 4748, 28, 7, 1986, 30, 469, 272, 2771, 5001, 94, 12539, 26, 30097, 1443, 440, 12166, 1846, 8321, 199, 93, 199, 199, 5645, 19293, 21815, 298, 35, 2, 4325, 949, 63, 3765, 18, 1547, 26, 399, 890, 12, 2771, 26, 11965, 469, 272, 4691, 485, 275, 2771, 14, 1045, 63, 1574, 8, 92, 439, 92, 2771, 2736, 5001, 94, 12539